Create a Hive enabled spark session, with a persisted derby metastore. Start Hive Thrift server to enable JDBC connections.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("SvnLocalSpark")\
    .config("spark.sql.warehouse.dir", "../spark-data")\
    .config("spark.hadoop.javax.jdo.option.ConnectionURL", "jdbc:derby:;databaseName=../hive_metastore_db;create=true")\
    .enableHiveSupport()\
    .getOrCreate()

sc = spark.sparkContext

print(f"spark {spark.version} {sc.uiWebUrl}")

spark 4.0.0 http://DESKTOP-4GOMK6M:4040


In [2]:
for db in spark.catalog.listDatabases():
    print(f"database: {db.name}")
    for tbl in spark.catalog.listTables(db.name):
        print(f"table:{tbl.name}")
print(f"current database:{spark.catalog.currentDatabase()}")

database: copytest
table:commercial_property
database: default
database: dwh_sales
table:dim_customer
table:dim_product
table:dim_store
table:fact_sales
database: landing
table:commercial_properties
database: sales_demo
table:calculations
table:customers
table:products
table:sales_details
table:sales_orders
table:stores
current database:default


In [3]:
# Start Hive Thrift Server
java_args = sc._gateway.new_array(sc._gateway.jvm.java.lang.String,2)
java_args[0]="dummy"
java_args[1]="false"

sc._jvm.org.apache.spark.sql.hive.thriftserver.HiveThriftServer2.main(java_args)

In [4]:
%load_ext sparksql_magic

In [7]:
%%sparksql
/* describe extended dwh_sales.fact_sales */
SELECT NOW()

now()
2025-06-02 15:48:35.845327
